## Software prototype for the Atmosphere project
This software prototype was developed as part of the Atmosphere project.  
It provides a workflow to analyze the potential utilization of waste heat through district heating in three steps:  
1. Cluster the building data into district heating clusters.  
2. Run an investment and operational optimization model to determine the optimal decisions.  
3. Visualize the results.

The following notebook demonstrates the use of its functionalities.

### Load the required packages 
The required Python packages for this notebook are loaded here.  
In addition, the developed libraries for the Atmosphere project are also loaded.  
An environment containing all necessary requirements and configurations is provided within the file `environment.yml`. It is recommended to execute all code within this environment to ensure proper operation.

In [ ]:
# import general packages
import os
import sys

# Ensure src is in the Python path
sys.path.append(os.path.join(os.getcwd(), "src"))

# import the Atmosphere libaries
from src import data, clustering, model, visualisation, constructive_elements

In [ ]:
# reload clustering 
import importlib
importlib.reload(clustering)

### Define a Scenario  

To start an analysis, the case study and scenario name must be defined. These are used as follows:  

- **Case study**: Defines the region of investigation, e.g., a specific city. Each case study requires an independent building dataset.  
- **Scenario**: Within each case study, multiple scenarios can be analyzed. Each scenario uses the same building dataset but with different parameters (e.g., costs, number of clusters, etc.).  

If a scenario name does not yet exist, a new folder for that scenario is created using default data. This data can subsequently be modified.  
If a scenario folder already exists, it is not overwritten.  

Additionally, the general configuration file `_config.yaml` contains the required general configurations.  
It is loaded and used in several functions throughout the script.  

In [ ]:
case_study = 'Puertollano'
scenario = 'Scenario_demo2'

config = data.load_config()
data.generate_new_scenario(case_study, scenario, config)

In [ ]:
# import clustering
from importlib import reload
reload(clustering)

### Cluster the Building Data  

Before processing the data, several input parameters and datasets can be configured in three files located in the `input` folder of each scenario:  

- **Parameters and costs**: These can be set in the file `input_ParameterCosts.xlsx`.  
  Note that all investment costs are annualized for one year (assuming one year of data for the optimization model).  

- **Heat generation units**: Data and settings for heat generation units can be specified in `input_HeatGenerationUnits.xlsx`.  
  - To add a unit, simply add a new row. All fields must be specified, and each unit can only be of one type (either a thermal energy storage, boiler, or waste heat unit).  
  - Costs can be set individually for each unit.  
  - The location, given in latitude/longitude, also determines the position and routing of the unit in the optimization model.  
  - To delete a unit, remove the corresponding row.  

- **Waste heat profiles**: These are defined in `input_WasteHeatProfiles.xlsx`.  
  - Each waste heat unit defined in `input_HeatGenerationUnits.xlsx` requires a corresponding time series for its waste heat profile.  
  - The name of the waste heat unit must match exactly.  
  - The profile must have the same temporal resolution and range as the heat demand time series.  

Subsequently, the building data is loaded and clustered according to the specified configuration.  
Several additional data processing steps are executed to prepare all data for the optimization model.  
The processed output data, which will be used by the optimization model, is then stored in the `data` folder.  


In [ ]:
# prepare the parameter file for the optimisation model 
clustering.prepare_parameter_file(case_study, scenario, config)

# load cost and parameter data
cost_parameter = clustering.load_cost_parameter(case_study, scenario, config)

# load the input data
buildings = clustering.read_geo_data_from_disk(case_study, config)
buildings_TS = clustering.read_building_TS_from_disk(case_study, config)
generation_units = clustering.read_heatGenUnits_from_disk(case_study, scenario, config)
waste_heat_profiles = clustering.read_wasteHeatProfiles_from_disk(generation_units, case_study, scenario, config)

# cluster the buildings
clustering.get_centroids(buildings)
building_cluster = clustering.cluster_buildings(buildings, cost_parameter)

# cluster the heat demand time series 
clustered_TS = clustering.cluster_heat_demand(building_cluster, buildings_TS)

# add hot water demand to the clustered time series
clustered_TS = clustering.add_hot_water_demand(building_cluster, clustered_TS, cost_parameter)

# merge clusters with generion units
all_heat_nodes = clustering.merge_heatGenUnits(building_cluster, generation_units)



In [ ]:
# create a network
network = clustering.propose_network(all_heat_nodes)

# save the data to the scenario folder for the optimisation model
clustering.save_data(all_heat_nodes,clustered_TS,network,generation_units,waste_heat_profiles, case_study, scenario, config)

In [ ]:
buildings_TS.columns

In [ ]:
buildings_TS.columns[1:].to_list()

In [ ]:
buildings_TS['building_1000'].plot()

#

In [ ]:
# visualisation of data
# If you want the visualise data in between to check the clustering or input data, you can use the explore function. 
# Here we plot the yearly demand of the buildings in the scenario.
buildings.explore('YearlyDemand', width=800, height=400, cmap='coolwarm')
# To explote different data you can change the first argument of the explore function.

In [ ]:
building_cluster.explore('YearlyDemand', width=800, height=400, cmap='coolwarm',marker_kwds={'radius': 10} )

In [ ]:
# plot building clusters but not the point, but the convex hull
building_cluster

In [ ]:
waste_heat_profiles.plot(x='hour', y='Electrolyser', title='Electrolyser Waste Heat Profile')

In [ ]:
clustered_TS.plot(x='hour', title='Clustered Heat Demand Profile', legend=False)

In [ ]:
network.plot()

In [ ]:
all_heat_nodes.explore('YearlyDemand', width=800, height=400, cmap='coolwarm',marker_kwds={'radius': 10} )

### Run the Optimization Model  

The optimization module loads the data, builds the model, solves it, and exports all the results to the `output` folder. Some ex-post calculations are done immediately and saved in the `ex-post` folder.  
The solver can be configured; more details can be found in the general description.  

The model itself is implemented as a mixed-integer linear program. Heat energy flow is modeled as a transportation problem between sources and heat demand nodes. The model aims to cover the entire heat demand while minimizing the total costs. Each building has its own heating system (existing system), and buildings may be connected to a district heating grid for defined investment costs.


In [ ]:
# reload the model.py file
from importlib import reload
reload(model)


In [ ]:
# run the combined optimisation model
model.run_model(case_study, scenario, config)


In [ ]:
model_input_data

### Results and visualisation
Key performance indicators, such as investment decisions and total cost shares, are calculated and can be accessed directly in the `expost` folder.  
For better representation, the results can be visualized, and some charts can be generated automatically. These are presented in the following section.  
All generated plots are automatically stored in the `plots` folder.  


In [ ]:
# read the model output
model_output_data = data.read_output_from_disk(case_study, scenario, config)
# read input data from the model
model_input_data = data.load_data_from_disk(case_study, scenario, config)

# extract the nodes and shapes from the model input data
dict_nodes = visualisation.extract_node_centroids(model_input_data)
dict_shapes = visualisation.extract_node_shapes(model_input_data)

# define the figure path
figure_path = os.path.join(case_study, config['scenario_dir'], scenario, config['plot_dir'])

# plot the investment decitions
visualisation.plot_investment_decisions(model_input_data, model_output_data, dict_nodes, dict_shapes, figure_path)

# plot the energy balance
visualisation.plot_energy_balance(visualisation.merge_time_series(model_input_data, model_output_data), figure_path)

# plot the time resolved energy balance; the time_inverall specifies the time resolution of the plot. 
# For example, 'h' for hourly, 'd' for daily, 'W' for weekly, 'M' for monthly, 'Y' for yearly.
visualisation.plot_time_resolved(visualisation.merge_time_series(model_input_data, model_output_data), figure_path, time_invervall='W')

In [ ]:
model_output_data

### Constructive elements

In [ ]:
# plot the investment decitions
gdf_BuildPipe = visualisation.plot_investment_decisions(model_input_data, model_output_data, dict_nodes, dict_shapes, figure_path)


In [ ]:
gdf_BuildPipe

In [ ]:
# Set the CRS if not already set (replace 25830 with your actual CRS if different)
#gdf_BuildPipe = gdf_BuildPipe.set_crs(epsg=25830, inplace=False)

gdf_BuildPipe_wgs84 = gdf_BuildPipe.set_crs(epsg=4326)

gdf_BuildPipe_wgs84.explore(
    'mass_flow',  # Optional: color lines by 'mass_flow'
    cmap='coolwarm',     # Choose colormap
    legend=True,         # Add a legend
    #tooltip=['node_from', 'node_to', 'mass_flow'],  # Show info on hover
    style_kwds={'weight': 4},  # Line thickness
    #tiles='OpenStreetMap', 
    height=500,
    width=800
)


In [ ]:
# Calculate the total length of built pipes
gdf_built_pipes = constructive_elements.total_length_of_built_pipes(gdf_BuildPipe)

print(gdf_built_pipes[['node_from', 'node_to', 'mass_flow', 'length_m']])

total_length_m = gdf_built_pipes['length_m'].sum()
print(f"Total length of built pipes: {total_length_m:.2f} meters")

In [ ]:
# Calculate the constructive elements of the built pipes
gdf_built_pipes_built = constructive_elements.design_pump_and_pipe(gdf_built_pipes)
print(gdf_built_pipes_built[['node_from', 'node_to', 'mass_flow', 'length_m', 'D_mm', 'ΔP_bar', 'Pump_electric_W']])

In [ ]:
# create rounded values for the diameter DN 100, 200,400 , 800, 1000
def round_to_nearest_dn(diameter_mm):
    dn_values = [100, 200, 400, 800, 1000]
    return min(dn_values, key=lambda x: abs(x - diameter_mm))

# Apply the rounding function to the diameter column
gdf_built_pipes_built['DN'] = gdf_built_pipes_built['D_mm'].apply(round_to_nearest_dn)

# group by DN, count pipes and sum the lengths
grouped = gdf_built_pipes_built.groupby('DN').agg(
    pipe_count=('length_m', 'count'),
    total_length_m=('length_m', 'sum')
).reset_index()
grouped

In [ ]:
model_output_data['vDHconnect']

In [ ]:
# access vDHconnect from the model output data
model_output_data['vDHconnect'].value.sum()